In [13]:
# Load packages we need
import sys
import os
import time

import numpy as np
import pandas as pd
import sklearn

import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.datasets import fashion_mnist, cifar10, imdb

from sklearn.model_selection import train_test_split

# import layers and callbacks we may use (may not be a complete list)
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Model, Sequential

from tensorflow.python.keras.utils import layer_utils


from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 14})

# Load the TensorBoard notebook extension
#%load_ext tensorboard


# Let's check our software versions
print('------------')
print('### Python version: ' + __import__('sys').version)
print('### NumPy version: ' + np.__version__)
print('### Scikit-learn version: ' + sklearn.__version__)
print('### Tensorflow version: ' + tf.__version__)
print('------------')

def var_exists(var_name):
    return (var_name in globals() or var_name in locals())
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array


------------
### Python version: 3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]
### NumPy version: 1.26.4
### Scikit-learn version: 1.4.1.post1
### Tensorflow version: 2.16.1
------------


In [2]:
# Load the csv

df = pd.read_csv("../data/train.csv")

# Create a list with all the images names
img_path_list = os.listdir("../data/train_images")

# Include a column in the dataframe with the path to the image
imgs_path = ("../data/train_images/")
for img in img_path_list:
    df["img_path"] = imgs_path + img
    

print(df["img_path"][0])
    
def load_image(img_path):
    img = Image.open(img_path).convert("RGB").resize((224, 224))
    img_array = np.array(img)  # Convert PIL Image to numpy array
    return img_array

# Apply the function to each row in the DataFrame to load the images
df["img_rgb"] = df["img_path"].apply(lambda x: load_image(x))
    
print(df.head())

../data/train_images/3150477025.jpg
         image_id  label                             img_path  \
0  1000015157.jpg      0  ../data/train_images/3150477025.jpg   
1  1000201771.jpg      3  ../data/train_images/3150477025.jpg   
2   100042118.jpg      1  ../data/train_images/3150477025.jpg   
3  1000723321.jpg      1  ../data/train_images/3150477025.jpg   
4  1000812911.jpg      3  ../data/train_images/3150477025.jpg   

                                             img_rgb  
0  [[[46, 65, 3], [58, 78, 7], [93, 118, 20], [13...  
1  [[[46, 65, 3], [58, 78, 7], [93, 118, 20], [13...  
2  [[[46, 65, 3], [58, 78, 7], [93, 118, 20], [13...  
3  [[[46, 65, 3], [58, 78, 7], [93, 118, 20], [13...  
4  [[[46, 65, 3], [58, 78, 7], [93, 118, 20], [13...  


In [5]:
all_x = np.array(df["img_rgb"])

print(type(all_x), all_x.shape)

all_y = np.array(df["label"])

print(type(all_y), all_y.shape)

<class 'numpy.ndarray'> (21397,)
<class 'numpy.ndarray'> (21397,)


In [7]:
all_x[0].shape

(224, 224, 3)

In [11]:
train_x, aux_x, train_y, aux_y = train_test_split(all_x, all_y, test_size=0.2, random_state=32)
test_x, val_x, test_y, val_y = train_test_split(aux_x, aux_y, test_size=0.5, random_state=32)



(224, 224, 3)


array([3, 3, 3, ..., 3, 3, 0])

In [19]:
def create_compile_cnn(input_shape=[224, 224, 3], num_outputs=5, verbose=False):
    
    model = keras.models.Sequential(name='CIFAR-10--CNN')
    
    model.add(Conv2D(32, kernel_size=(16,16), input_shape=input_shape, activation='relu', 
              padding='same', strides=(1,1), kernel_initializer='lecun_uniform',  name='conv1'))
    model.add(Conv2D(32, kernel_size=(16,16), activation = 'relu', 
              padding='same', strides=(1,1), kernel_initializer='lecun_uniform',  name='conv2'))
    model.add(MaxPooling2D((2,2), name='MaxPool1'))
    
    model.add(Conv2D(64, kernel_size=(16,16), activation = 'relu', 
              padding='same', strides=(1,1), kernel_initializer='lecun_uniform',  name='conv3'))
    model.add(Conv2D(64, kernel_size=(16,16), activation = 'relu', 
              padding='same', strides=(1,1), kernel_initializer='lecun_uniform',  name='conv4'))
    model.add(MaxPooling2D((2,2), name='MaxPool2'))
    
    model.add(Conv2D(128, kernel_size=(16,16), activation = 'relu', 
              padding='same', strides=(1,1), kernel_initializer='lecun_uniform',  name='conv5'))
    model.add(Conv2D(128, kernel_size=(16,16), activation = 'relu', 
              padding='same', strides=(1,1), kernel_initializer='lecun_uniform',  name='conv6'))
    model.add(MaxPooling2D((2,2), name='MaxPool3'))
    
    model.add(Flatten(name='flatten'))
    
    model.add(Dense(128, activation='relu', kernel_initializer='lecun_uniform', name='dense1'))
    model.add(Dropout(0.25, name='drop1'))
    
    model.add(Dense(96, activation='relu', kernel_initializer='lecun_uniform', name='dense2'))
    model.add(Dropout(0.25, name='drop2'))
    
    model.add(Dense(num_outputs, activation="softmax", name='output'))
    
    opt = keras.optimizers.Adam(learning_rate=0.002)
    
    if verbose:
        model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

In [23]:
model = create_compile_cnn(verbose=True)

Model: "CIFAR-10--CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 224, 224, 32)   │        24,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 224, 224, 32)   │       262,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool1 (MaxPooling2D)         │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 112, 112, 64)   │       524,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 112, 112, 64)   │     1,048,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool2 (MaxPooling2D)         │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv5 (Conv2D)                  │ (None, 56, 56, 128)    │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv6 (Conv2D)                  │ (None, 56, 56, 128)    │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool3 (MaxPooling2D)         │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop1 (Dropout)                 │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 96)             │        12,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop2 (Dropout)                 │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 5)              │           485 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,009,541 (80.15 MB)

 Trainable params: 21,009,541 (80.15 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
early_stop_cb = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
max_epochs = 5
batch_size = 64
    
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=max_epochs, batch_size=batch_size, 
                         shuffle=True, callbacks=[early_stop_cb])
    

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).